## Sales Forecasting


### Reg No: IT21127946
### Name: Christy H.M

<hr/>

<ul>
    <li><b>Target Variable:</b> Total sales (considering the Total attribute)</li>
    <li><b>Predictors:</b> Branch, Customer type, Gender, Product line, Quantity, Date, Time, Payment, COGS, and Gross income</li>
    <li><b>Objective:</b> Predict the total sales amount based on various factors such as the type of products, customer demographics, and purchase details.</li>
</ul>




In [176]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder

In [177]:
dataset = pd.read_csv('../dataset/SuperStoreOrders.csv')

In [178]:
dataset

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.1400,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.0360,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.6400,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.0550,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.7700,4.70,Medium,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,CA-2014-115427,31-12-2014,4/1/2015,Standard Class,Erica Bern,Corporate,California,United States,US,West,...,Office Supplies,Binders,"Cardinal Slant-D Ring Binder, Heavy Gauge Vinyl",14,2,0.2,4.5188,0.89,Medium,2014
51286,MO-2014-2560,31-12-2014,5/1/2015,Standard Class,Liz Preis,Consumer,Souss-Massa-Draâ,Morocco,Africa,Africa,...,Office Supplies,Binders,"Wilson Jones Hole Reinforcements, Clear",4,1,0.0,0.4200,0.49,Medium,2014
51287,MX-2014-110527,31-12-2014,2/1/2015,Second Class,Charlotte Melton,Consumer,Managua,Nicaragua,LATAM,Central,...,Office Supplies,Labels,"Hon Color Coded Labels, 5000 Label Set",26,3,0.0,12.3600,0.35,Medium,2014
51288,MX-2014-114783,31-12-2014,6/1/2015,Standard Class,Tamara Dahlen,Consumer,Chihuahua,Mexico,LATAM,North,...,Office Supplies,Labels,"Hon Legal Exhibit Labels, Alphabetical",7,1,0.0,0.5600,0.20,Medium,2014


In [179]:
# Check for missing values
print("\nMissing values:")
print(dataset.isnull().sum())


Missing values:
order_id          0
order_date        0
ship_date         0
ship_mode         0
customer_name     0
segment           0
state             0
country           0
market            0
region            0
product_id        0
category          0
sub_category      0
product_name      0
sales             0
quantity          0
discount          0
profit            0
shipping_cost     0
order_priority    0
year              0
dtype: int64


In [180]:
# Identify categorical variables
categorical_columns = dataset.select_dtypes(include=['object']).columns

In [181]:
categorical_columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name',
       'segment', 'state', 'country', 'market', 'region', 'product_id',
       'category', 'sub_category', 'product_name', 'sales', 'order_priority'],
      dtype='object')

In [182]:
# Selecting relevant columns
selected_columns = ['segment', 'category', 'discount','shipping_cost', 'order_priority', 'ship_mode','market', 'state', 'sub_category', 'country', 'sales']

In [183]:
# Creating a new DataFrame with selected columns
preprocessed_dataset = dataset[selected_columns].copy()

In [184]:
#Checking preprocessed dataframe
preprocessed_dataset

,segment,category,discount,shipping_cost,order_priority,ship_mode,market,state,sub_category,country,sales
0,Consumer,Office Supplies,0.0,35.46,Medium,Standard Class,Africa,Constantine,Storage,Algeria,408
1,Consumer,Office Supplies,0.1,9.72,Medium,Standard Class,APAC,New South Wales,Supplies,Australia,120
2,Consumer,Office Supplies,0.0,8.17,High,Second Class,EMEA,Budapest,Storage,Hungary,66
3,Home Office,Office Supplies,0.5,4.82,High,Second Class,EU,Stockholm,Paper,Sweden,45
4,Consumer,Furniture,0.1,4.70,Medium,Standard Class,APAC,New South Wales,Furnishings,Australia,114
...,...,...,...,...,...,...,...,...,...,...,...
51285,Corporate,Office Supplies,0.2,0.89,Medium,Standard Class,US,California,Binders,United States,14
51286,Consumer,Office Supplies,0.0,0.49,Medium,Standard Class,Africa,Souss-Massa-Draâ,Binders,Morocco,4
51287,Consumer,Office Supplies,0.0,0.35,Medium,Second Class,LATAM,Managua,Labels,Nicaragua,26
51288,Consumer,Office Supplies,0.0,0.20,Medium,Standard Class,LATAM,Chihuahua,Labels,Mexico,7


In [185]:
# Handling missing values (if any)
preprocessed_dataset.dropna(inplace=True) 

In [186]:
#Preprocessed data
preprocessed_dataset

,segment,category,discount,shipping_cost,order_priority,ship_mode,market,state,sub_category,country,sales
0,Consumer,Office Supplies,0.0,35.46,Medium,Standard Class,Africa,Constantine,Storage,Algeria,408
1,Consumer,Office Supplies,0.1,9.72,Medium,Standard Class,APAC,New South Wales,Supplies,Australia,120
2,Consumer,Office Supplies,0.0,8.17,High,Second Class,EMEA,Budapest,Storage,Hungary,66
3,Home Office,Office Supplies,0.5,4.82,High,Second Class,EU,Stockholm,Paper,Sweden,45
4,Consumer,Furniture,0.1,4.70,Medium,Standard Class,APAC,New South Wales,Furnishings,Australia,114
...,...,...,...,...,...,...,...,...,...,...,...
51285,Corporate,Office Supplies,0.2,0.89,Medium,Standard Class,US,California,Binders,United States,14
51286,Consumer,Office Supplies,0.0,0.49,Medium,Standard Class,Africa,Souss-Massa-Draâ,Binders,Morocco,4
51287,Consumer,Office Supplies,0.0,0.35,Medium,Second Class,LATAM,Managua,Labels,Nicaragua,26
51288,Consumer,Office Supplies,0.0,0.20,Medium,Standard Class,LATAM,Chihuahua,Labels,Mexico,7


In [187]:
# Encoding categorical variables using one-hot encoding
# preprocessed_dataset = pd.get_dummies(preprocessed_dataset, columns=['segment','category','order_priority','ship_mode','market'])

labelEncoder = LabelEncoder()
preprocessed_dataset['segment'] = labelEncoder.fit_transform(preprocessed_dataset['segment'])
preprocessed_dataset['category'] = labelEncoder.fit_transform(preprocessed_dataset['category'])
preprocessed_dataset['order_priority'] = labelEncoder.fit_transform(preprocessed_dataset['order_priority'])
preprocessed_dataset['ship_mode'] = labelEncoder.fit_transform(preprocessed_dataset['ship_mode'])
preprocessed_dataset['market'] = labelEncoder.fit_transform(preprocessed_dataset['market'])
preprocessed_dataset['state'] = labelEncoder.fit_transform(preprocessed_dataset['state'])
preprocessed_dataset['sub_category'] = labelEncoder.fit_transform(preprocessed_dataset['sub_category'])
preprocessed_dataset['country'] = labelEncoder.fit_transform(preprocessed_dataset['country'])

In [188]:
# Remove non-numeric characters (e.g., commas) and convert 'sales' column to float
preprocessed_dataset['sales'] = preprocessed_dataset['sales'].str.replace(',', '').astype(float)

In [189]:
preprocessed_dataset

,segment,category,discount,shipping_cost,order_priority,ship_mode,market,state,sub_category,country,sales
0,0,1,0.0,35.46,3,3,1,255,14,2,408.0
1,0,1,0.1,9.72,3,3,0,702,15,6,120.0
2,0,1,0.0,8.17,1,2,3,175,14,56,66.0
3,2,1,0.5,4.82,1,2,4,939,12,123,45.0
4,0,0,0.1,4.70,3,3,0,702,9,6,114.0
...,...,...,...,...,...,...,...,...,...,...,...
51285,1,1,0.2,0.89,3,3,6,192,3,139,14.0
51286,0,1,0.0,0.49,3,3,1,923,3,85,4.0
51287,0,1,0.0,0.35,3,2,5,599,10,92,26.0
51288,0,1,0.0,0.20,3,3,5,230,10,81,7.0


In [190]:
# Split features (X) and target variable (y)
X = preprocessed_dataset.drop(columns=['sales'])
y = preprocessed_dataset['sales']

In [191]:
print(preprocessed_dataset.dtypes)


segment             int32
category            int32
discount          float64
shipping_cost     float64
order_priority      int32
ship_mode           int32
market              int32
state               int32
sub_category        int32
country             int32
sales             float64
dtype: object


In [192]:
# Use the `to_numeric()` function to convert the column to floats
# preprocessed_dataset['sales'] = pd.to_numeric(preprocessed_dataset['sales'], errors='coerce')


In [193]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [194]:
from sklearn.linear_model import LinearRegression

# 4. Model Selection
model = LinearRegression()  # Initialize the linear regression model

# 5. Model Training
model.fit(X_train, y_train)  # Train the model using the training data


LinearRegression()

In [195]:
# Make predictions on the testing set
y_pred = model.predict(X_test)

In [196]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [197]:
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

Mean Squared Error (MSE): 93698.9008841255
Mean Absolute Error (MAE): 124.61110193773668
R-squared (R2): 0.6169930422016415


In [171]:
from sklearn.tree import DecisionTreeRegressor

# Initialize the Decision Tree Regressor model
decision_tree_model = DecisionTreeRegressor(random_state=42)

# Train the Decision Tree Regressor model
decision_tree_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_decision_tree = decision_tree_model.predict(X_test)

# Evaluate the Decision Tree Regressor model
mse_decision_tree = mean_squared_error(y_test, y_pred_decision_tree)
mae_decision_tree = mean_absolute_error(y_test, y_pred_decision_tree)
r2_decision_tree = r2_score(y_test, y_pred_decision_tree)

print("Decision Tree Regressor Metrics:")
print("Mean Squared Error (MSE):", mse_decision_tree)
print("Mean Absolute Error (MAE):", mae_decision_tree)
print("R-squared (R2):", r2_decision_tree)


Decision Tree Regressor Metrics:
Mean Squared Error (MSE): 153884.62651101578
Mean Absolute Error (MAE): 104.7328426593878
R-squared (R2): 0.3709757308166437


In [172]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the Random Forest Regressor model
random_forest_model = RandomForestRegressor(random_state=42)

# Train the Random Forest Regressor model
random_forest_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_random_forest = random_forest_model.predict(X_test)

# Evaluate the Random Forest Regressor model
mse_random_forest = mean_squared_error(y_test, y_pred_random_forest)
mae_random_forest = mean_absolute_error(y_test, y_pred_random_forest)
r2_random_forest = r2_score(y_test, y_pred_random_forest)

print("Random Forest Regressor Metrics:")
print("Mean Squared Error (MSE):", mse_random_forest)
print("Mean Absolute Error (MAE):", mae_random_forest)
print("R-squared (R2):", r2_random_forest)


Random Forest Regressor Metrics:
Mean Squared Error (MSE): 73351.51792706331
Mean Absolute Error (MAE): 80.66445660529762
R-squared (R2): 0.7001657280283424


In [173]:
from sklearn.svm import SVR

# Initialize the SVR model
svr_model = SVR()

# Train the SVR model
svr_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_svr = svr_model.predict(X_test)

# Evaluate the SVR model
mse_svr = mean_squared_error(y_test, y_pred_svr)
mae_svr = mean_absolute_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print("Support Vector Regression (SVR) Metrics:")
print("Mean Squared Error (MSE):", mse_svr)
print("Mean Absolute Error (MAE):", mae_svr)
print("R-squared (R2):", r2_svr)


Support Vector Regression (SVR) Metrics:
Mean Squared Error (MSE): 176971.7814785465
Mean Absolute Error (MAE): 157.8829051360065
R-squared (R2): 0.27660385553425915


In [198]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the Gradient Boosting Regressor model
gradient_boosting_model = GradientBoostingRegressor(random_state=42)

# Train the Gradient Boosting Regressor model
gradient_boosting_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_gradient_boosting = gradient_boosting_model.predict(X_test)

# Evaluate the Gradient Boosting Regressor model
mse_gradient_boosting = mean_squared_error(y_test, y_pred_gradient_boosting)
mae_gradient_boosting = mean_absolute_error(y_test, y_pred_gradient_boosting)
r2_gradient_boosting = r2_score(y_test, y_pred_gradient_boosting)

print("Gradient Boosting Regressor Metrics:")
print("Mean Squared Error (MSE):", mse_gradient_boosting)
print("Mean Absolute Error (MAE):", mae_gradient_boosting)
print("R-squared (R2):", r2_gradient_boosting)


Gradient Boosting Regressor Metrics:
Mean Squared Error (MSE): 64511.28191897883
Mean Absolute Error (MAE): 75.40911272920074
R-squared (R2): 0.736301390962779


In [175]:
from sklearn.linear_model import Ridge

# Initialize the Ridge Regression model
ridge_model = Ridge(alpha=1.0, random_state=42)  # You can adjust the regularization strength (alpha) as needed

# Train the Ridge Regression model
ridge_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred_ridge = ridge_model.predict(X_test)

# Evaluate the Ridge Regression model
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("Ridge Regression Metrics:")
print("Mean Squared Error (MSE):", mse_ridge)
print("Mean Absolute Error (MAE):", mae_ridge)
print("R-squared (R2):", r2_ridge)


Ridge Regression Metrics:
Mean Squared Error (MSE): 93698.93176982299
Mean Absolute Error (MAE): 124.6105205053461
R-squared (R2): 0.6169929159521665
